In [198]:
import numpy as np
import PIL
import matplotlib.pyplot as plt  
import matplotlib.image as im   
from scipy import ndimage
import cv2 as cv
%matplotlib inline


filename = "P1.JPG"
src_image = cv.imread(filename)



def gris_moyen(image):
    cop = image.copy()
    gris = cv.cvtColor(cop, cv.COLOR_BGR2GRAY)
    return gris

def affiche_image(name, src):

    cv.imshow(name, src)
    cv.waitKey(0)
    cv.destroyAllWindows()


def print_corners(corner_list, image_source):
    for indices in corner_list :
        #cv.circle(image_source, (indices[0], indices[1]), 5, (0,0,255), 2)
        image_source[indices[1], indices[0]] = [0,0,255]
        #indices[1] = ligne
        #indices[0] = colonne

    affiche_image("corners", image_source)
    

def rotate_image(src):
    cop = src.copy()
    rotated = cv.rotate(cop, cv.ROTATE_90_CLOCKWISE)
    return rotated


#affiche_image("original", rotate_image(src_image))

img_gray = gris_moyen(src_image)


In [199]:
#On calcule les matrice gradient selon x et y convolution avec masque : a tester avec plusieurs masques
masqueX = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
masqueY =np.array([[-1,-2,-1], [0, 0, 0], [1, 2, 1]])
masque_gaussien = np.array([[1,2,1],[2,4,2],[1,2,1]])/16



Ix = cv.filter2D(img_gray, -1, masqueX)
Iy = cv.filter2D(img_gray, -1, masqueY)

Ix_carre = np.square(Ix)
Iy_carre = np.square(Iy)
Ixy = np.multiply(Ix,Iy)

**EXERCICE 1: DETECTION DE POINTS D’INTERET**

**Detecteur de Harris**

- fenetre rectangulaire


In [200]:



def suppression_non_maxima(C_harris, list_corners):
     #apres calcul de C et de la liste des coins
     #on regarde le voisinnage de chaque coin et on garde le max

    print(" AVANT SUPPR : len list_corners = ", len(list_corners))
    
    taille = 300
    for indices in list_corners :
        x = indices[0]
        y = indices[1]


        if x < taille or y < taille or x > C_harris.shape[1] - taille or y > C_harris.shape[0] - taille:
            pass
        
        else :
            start_y = y - taille
            end_y = y + taille

            start_x = x - taille
            end_x = x + taille

            window = C_harris[start_y : end_y, start_x : end_x]
            max = window.max()

            if max != C_harris[y, x]:
                list_corners.remove(indices)
        
    new_list = list_corners
    print(" APRES SUPPR : len list_corners = ", len(new_list))
    
    return new_list
     

def Harris_rectangle(img_src): 
    offset = 1 #deplacement de la fenetre : le meme selon x et y

    corner_list = []
    img_gray = gris_moyen(img_src)
    
    y_end = img_gray.shape[0] - offset
    x_end = img_gray.shape[1] - offset


    for x in range(offset, x_end):
            for y in range(offset, y_end):

                

                
                start_y = y - offset
                end_y = y + offset + 1
                
                start_x = x - offset
                end_x = x + offset + 1

                windowIxx = Ix_carre[start_y : end_y, start_x : end_x]
                windowIxy = Ixy[start_y : end_y, start_x : end_x]
                windowIyy = Iy_carre[start_y : end_y, start_x : end_x]

                Somme_xx = windowIxx.sum()
                Somme_xy = windowIxy.sum()
                Somme_yy = windowIyy.sum()


                det = (Somme_xx * Somme_yy) - (Somme_xy**2)
                trace = Somme_xx + Somme_yy

                C = det - 0.06 * (trace**2)

                if C > 100000: #mieux
                    corner_list.append([x, y])

    

    
    #Afficahge des coins
    image_apres_Harris_recrangulaire = img_src.copy()
    print_corners(corner_list, image_apres_Harris_recrangulaire)







In [201]:
#test invariance rotation
image_apres_rotation = rotate_image(src_image)


Harris_rectangle(src_image)
Harris_rectangle(image_apres_rotation)


(c) ´Etudier l’influence du type de fenetre de ponderation, de sa taille ainsi que du parametre k dans la detection de point
d’interet.

En utilisant simplement une fenetre rectangulaire, on est pas invariant en rotation.
voyons en pondérant avec une gaussienne :

In [202]:
#Ix, Iy, Ixy
#Ix_carre, Iy_carre, Ixy


def Harris_gaussienne(src):

    grey = gris_moyen(src)

    #on les convolutionne avec un masque gaussien
    Ix_carre_gaussienn = cv.filter2D(Ix_carre, -1, masque_gaussien)
    Iy_carre_gaussien = cv.filter2D(Iy_carre, -1, masque_gaussien)
    Ixy_gaussien = cv.filter2D(Ixy, -1, masque_gaussien)

    liste_corners_gaussienne = []

    for y in range(1, grey.shape[0] - 1):
        for x in range(1, grey.shape[1] - 1):
            
            Matrx = np.array([[Ix_carre_gaussienn[y,x], Ixy_gaussien[y,x]],[Ixy_gaussien[y,x], Iy_carre_gaussien[y,x]]])
            det = np.linalg.det(Matrx)
            trace = np.trace(Matrx)
            C = det - 0.04 * (trace**2)

            if C > 5800:
                liste_corners_gaussienne.append([x, y])
                cv.circle(src, (x, y), 1, (0,0,255), 1)
    

    print("before removing, there are ", len(liste_corners_gaussienne), " corners")
    liste_corners_gaussienne = suppression_non_maxima(Ix_carre_gaussienn, liste_corners_gaussienne)
    print("after removing, there are ", len(liste_corners_gaussienne), " corners")
    affiche_image("with removing", src)

    return liste_corners_gaussienne



    



In [203]:

image_apres_Harris_gaussien = src_image.copy()
liste_corners_gaussienne = Harris_gaussienne(image_apres_Harris_gaussien)
print_corners(liste_corners_gaussienne, image_apres_Harris_gaussien)


before removing, there are  1003  corners
 AVANT SUPPR : len list_corners =  1003
 APRES SUPPR : len list_corners =  1003
after removing, there are  1003  corners


In [204]:

#Test avec fonction déja implémentés

img = cv.imread(filename)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
gray = np.float32(gray)
corners = cv.cornerHarris(gray,2,3,0.04)

img[corners>0.01*corners.max()]=[0,0,255]
cv.imshow('corners',img)

if cv.waitKey(0) & 0xff == 27:
 cv.destroyAllWindows()

#coordonnées des coins
corners = np.argwhere(corners>0.01*corners.max())



**Detecteur Fast**


In [205]:
#fast detector

def circle_around_pixel(img, i, j, seuil):
    #rend une liste des 16 pixels autour du pixel (i,j)
    # dans un rayon de 3 pixels

    list_pixels = []
    
    list_pixels.append(img[i-3][j])
    list_pixels.append(img[i-3][j+1])
    list_pixels.append(img[i-2][j+2])
    list_pixels.append(img[i-1][j+3
                                ])
    list_pixels.append(img[i][j+3])
    list_pixels.append(img[i+1][j+3])
    list_pixels.append(img[i+2][j+2])
    list_pixels.append(img[i+3][j+1])

    list_pixels.append(img[i+3][j])
    list_pixels.append(img[i+3][j-1])
    list_pixels.append(img[i+2][j-2])
    list_pixels.append(img[i+1][j-3])

    list_pixels.append(img[i][j-3])
    list_pixels.append(img[i-1][j-3])
    list_pixels.append(img[i-2][j-2])
    list_pixels.append(img[i-3][j-1])


    ref = img[i][j]

    for k in range(len(list_pixels)):
        if list_pixels[k]<ref-seuil or list_pixels[k]>ref+seuil:
            list_pixels[k]=1
        else:
            list_pixels[k]=0
    

    return list_pixels



def count_successive_one_in_a_list(list_pixels):
    #pn a 16 pixels
    count = 0
    flag_list = False
    list_max_count = []
    for i in range(len(list_pixels)):
        if list_pixels[i]==1:
            count+=1
            flag_list = True
        else:
            if flag_list:
                break
    return count


            

#en mm temps qu'on calcule C, on calculte un score pour chaque pixel et on seuil

#on regarde 8*8

In [206]:



def trouver_n_1_successifs(vecteur, n):
    vecteur_convolution = np.ones(n, dtype=int)
    
    resultat_convolution = np.convolve(vecteur, vecteur_convolution, mode='valid')
    indices = np.where(resultat_convolution >= n)[0]
    
    if len(indices) > 0:
        return True 
    else:
        return False  
    


image_apres_Fast = src_image.copy()


def Fast_detector(img, seuil):
    for i in range(3, img.shape[0]-3):
        for j in range(3, img.shape[1]-3):
            list_pixels = circle_around_pixel(img, i, j, seuil)
            flag = trouver_n_1_successifs(list_pixels, 10)
            if flag:
                image_apres_Fast[i][j]=[0,0,255]
    return image_apres_Fast

image_apres_Fast = Fast_detector(img_gray, 10)
affiche_image("Fast", image_apres_Fast)